In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
image_size = [224,224]
data_path = 'Data'

In [3]:
vgg = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)

In [4]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [5]:
x = vgg.output
x = GlobalAveragePooling2D()(x)

In [6]:
x = Dense(1024,activation='relu')(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)

In [7]:
preds = Dense(2,activation='softmax')(x)

In [8]:
model = Model(inputs = vgg.input,outputs=preds)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
for layer in vgg.layers:
    layer.trainable = False

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory paths for training and testing data
train_data_dir = 'Data/train/'
test_data_dir = 'Data/test/'

# Image data generators for training and testing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for testing data
)

Found 3329 images belonging to 2 classes.
Found 819 images belonging to 2 classes.


In [15]:
from tensorflow.keras.optimizers import Adam

# Compile the model with an optimizer, loss function, and metrics
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',  # Adjust the loss function based on your task
    metrics=['accuracy']  # Adjust metrics based on your requirements
)

# Calculate steps per epoch for training data
step_size_train = train_generator.n // train_generator.batch_size

# Train the model using the training data generator
history = model.fit(
    train_generator,
    steps_per_epoch=step_size_train,
    epochs=5,
    validation_data=test_generator,
    validation_steps=test_generator.n // test_generator.batch_size
)

Epoch 1/5
104/104 [==============================] - 1200s 12s/step - loss: 0.9290 - accuracy: 0.7540 - val_loss: 0.4045 - val_accuracy: 0.7837
Epoch 2/5
104/104 [==============================] - 1374s 13s/step - loss: 0.3210 - accuracy: 0.8429 - val_loss: 0.5686 - val_accuracy: 0.7950
Epoch 3/5
104/104 [==============================] - 1565s 15s/step - loss: 0.2556 - accuracy: 0.8741 - val_loss: 0.9398 - val_accuracy: 0.8037
Epoch 4/5
104/104 [==============================] - 1215s 12s/step - loss: 0.2418 - accuracy: 0.8926 - val_loss: 0.4977 - val_accuracy: 0.8325
Epoch 5/5
104/104 [==============================] - 1123s 11s/step - loss: 0.1824 - accuracy: 0.9245 - val_loss: 0.8792 - val_accuracy: 0.8125


In [17]:
from tensorflow.keras.models import load_model

In [18]:
model.save('dementia.h5')